# First look at first model

In [1]:
from gensim.models import FastText

In [2]:
# The load() method must be called on the FastText class, rather than on an instance
model = FastText.load('models/last-15-years-25-epochs.ftmodel')

In [38]:
sims = model.wv.most_similar(positive=['romantic'], topn=50)
i = 0
for sim in sims:
    i += 1
    print(f'{i}. {sim[0]}, dist = {sim[1]}')

1. neoromantic, dist = 0.957636833190918
2. antiromantic, dist = 0.9532189965248108
3. metaromantic, dist = 0.9492950439453125
4. preromantic, dist = 0.9464113712310791
5. necromantic, dist = 0.9446543455123901
6. nonromantic, dist = 0.9406825304031372
7. postromantic, dist = 0.9399392604827881
8. hyperromantic, dist = 0.9314874410629272
9. unromantic, dist = 0.9279868602752686
10. romanticist, dist = 0.9219558238983154
11. romantická, dist = 0.9128398895263672
12. Unromantic, dist = 0.9121805429458618
13. romantice, dist = 0.9095699787139893
14. romantickým, dist = 0.9052660465240479
15. romanticise, dist = 0.9040356278419495
16. romantický, dist = 0.9036016464233398
17. romanticus, dist = 0.9010462760925293
18. romantick, dist = 0.8986073732376099
19. romanticising, dist = 0.8957149982452393
20. romanticism, dist = 0.8934667706489563
21. ofRomantic, dist = 0.89158034324646
22. sentimentalist, dist = 0.8909026384353638
23. romanticizing, dist = 0.8873521685600281
24. sentimental, dist

In [19]:
sims = model.wv.most_similar(positive=['self','control'], topn=50)
i = 0
for sim in sims:
    i += 1
    print(f'{i}. {sim[0]}, dist = {sim[1]}')

1. selfness, dist = 0.8833435773849487
2. selfconscious, dist = 0.8673973083496094
3. self77, dist = 0.8636106252670288
4. selfs, dist = 0.861426591873169
5. subjectification, dist = 0.8501474261283875
6. controll, dist = 0.8486688733100891
7. possessiveness, dist = 0.8426641225814819
8. individualization, dist = 0.8422468304634094
9. repressiveness, dist = 0.841666579246521
10. controlling, dist = 0.8411588668823242
11. protectiveness, dist = 0.8407149910926819
12. subjection, dist = 0.8393940925598145
13. objectification, dist = 0.8385723233222961
14. personalization, dist = 0.8370845913887024
15. controle, dist = 0.8363387584686279
16. selflessness, dist = 0.8361494541168213
17. controlar, dist = 0.8352086544036865
18. individuality, dist = 0.8333074450492859
19. personality, dist = 0.8332066535949707
20. determination, dist = 0.8329977989196777
21. controul, dist = 0.8323333263397217
22. conscientiousness, dist = 0.8315858840942383
23. controllo, dist = 0.8314959406852722
24. nonse

In [17]:
from nltk.tokenize import wordpunct_tokenize
wordpunct_tokenize('self-development')

['self', '-', 'development']

# Hmmm...

It doesn't seem to be throwing up anything much interesting. We could try comparing two corpora, one where 'romanticism' is a key term, and one where it isn't...

In [1]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from utils import JSTORCorpus

In [2]:
corpus = JSTORCorpus(data_dir='data/ocr', meta_dir='data/metadata')

Parsing xml files in data/metadata. Associated .txt in data/ocr


  0%|          | 0/54017 [00:00<?, ?it/s]/shared/home/mgf/romantic-self/utils.py:83: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 83 of the file /shared/home/mgf/romantic-self/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  meta_xml = BeautifulSoup(file.read())
100%|██████████| 54017/54017 [11:24<00:00, 78.88it/s] 

44802 documents parsed successfully. 9215 documents skipped.


In [5]:
idx_doc = {}
i = 0
for doc in corpus.corpus_meta:
    idx_doc[i] = doc
doc_idx = {val:key for key,val in idx_doc.items()}

In [6]:
dct = Dictionary(corpus.iter_lower)
dct.save('models/tfidf-lower-dct')

In [8]:
bow_corpus = (dct.doc2bow(text) for text in corpus.iter_lower) # Single-use generator
model = TfidfModel(bow_corpus)
model.save('models/tfidf-lower-model')

In [34]:
dct.token2id['Enlightenment']

769

In [21]:
# Now apply the model to documents from the corpus...
bow_corpus = (dct.doc2bow(text) for text in corpus)
tfidf_corpus = model[bow_corpus]
romantic_articles = []
for doc in tfidf_corpus:
    

[(0, 0.000512341311660116), (1, 0.08062063130933346), (2, 0.07540289823456944), (3, 0.08062063130933346), (4, 0.07540289823456944), (5, 0.07540289823456944), (6, 0.07235072004706), (7, 0.08062063130933346), (8, 0.08062063130933346), (9, 0.07018516515980544), (10, 0.06597260267991195), (11, 0.08062063130933346), (12, 0.06408080878478654), (13, 0.08062063130933346), (14, 0.08062063130933346), (15, 0.07235072004706), (16, 0.07018516515980544), (17, 0.07540289823456944), (18, 0.06850543029141587), (19, 0.07235072004706), (20, 0.07235072004706), (21, 0.05974969901027741), (22, 0.07235072004706), (23, 6.939736438041324e-05), (24, 0.026769448927391193), (25, 0.000638255034507209), (26, 3.9319133204741936e-05), (27, 0.16104245243547055), (28, 0.004060661890514166), (29, 0.022842209399052217), (30, 0.02293004700750414), (31, 0.0233853004470553), (32, 0.0030172168072851654), (33, 0.0006225221084324688), (34, 0.01845642539608681), (35, 0.05163826934121961), (36, 0.02800079303162913), (37, 0.02158

In [40]:
'FOO'.lower()

'foo'

## Have a look at topic model

In [2]:
from gensim.models import LdaModel
model = LdaModel.load('models/corpus-lower-lda')

In [3]:
from gensim.corpora import Dictionary
corp_dict = Dictionary.load('models/corpus-lower-dct')

In [10]:
corp_dict.token2id

{'(\uf645\uf643\uf643\uf648),': 0,
 '(\uf645\uf643\uf643\uf649),': 1,
 '(\uf645\uf643\uf644\uf643),': 2,
 '(\uf645\uf643\uf644\uf644),': 3,
 ',”': 4,
 '.”': 5,
 '420': 6,
 '421': 7,
 '422': 8,
 'abridgement': 9,
 'achievement': 10,
 'action': 11,
 'addition': 12,
 'aesthetics': 13,
 'affective': 14,
 'agency': 15,
 'allegiance': 16,
 'america': 17,
 'annotated': 18,
 'annual': 19,
 'appear': 20,
 'appeared': 21,
 'approaches': 22,
 'arthur': 23,
 'articles': 24,
 'assistant': 25,
 'associate': 26,
 'associations': 27,
 'average': 28,
 'award': 29,
 'b': 30,
 'ballads': 31,
 'bands': 32,
 'baylor': 33,
 'became': 34,
 'benford': 35,
 'blackwell': 36,
 'booklength': 37,
 'borderlines': 38,
 'boucicault': 39,
 'bristow': 40,
 'britain': 41,
 'british': 42,
 'burlesque': 43,
 'centennial': 44,
 'chapter': 45,
 'charles': 46,
 'chuzzlewit': 47,
 'class': 48,
 'coauthor': 49,
 'coedited': 50,
 'coediting': 51,
 'coeditor': 52,
 'cognition': 53,
 'cognitive': 54,
 'collection': 55,
 'college'

In [16]:
from nltk.corpus import stopwords

In [18]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each